In [1]:
import pandas as pd
coord_file_path = 'adk-lakes-centroids-centerextracted.csv'
df_coord = pd.read_csv(coord_file_path)
df_coord.dropna(inplace=True)
df_coord

,PERMANENT_ID,GNIS_Name,AreaSqKm,Elevation,ReachCode,FType,NHDPlusID,area_ha,Lon,Lat
2,89362545,Challis Pond,0.065000,350.5,2.020000e+12,390,1.000000e+13,6.500000,-73.670123,43.984701
3,89361919,Bradley Pond,0.039364,0.0,2.020000e+12,390,1.000000e+13,3.936431,-74.110881,44.096109
27,47722313,Warner Lake,0.026000,0.0,2.020000e+12,390,1.000000e+13,2.600000,-74.563007,43.429974
29,47722092,Charley Lake,0.154000,0.0,2.020000e+12,390,1.000000e+13,15.400000,-74.318671,43.441170
45,89362163,Triangle Pond,0.021000,375.0,2.020000e+12,390,1.000000e+13,2.100000,-73.650409,44.039809
...,...,...,...,...,...,...,...,...,...,...
11842,132876274,Twin Ponds,0.091953,533.4,4.300000e+12,390,6.000000e+13,9.195258,-73.965560,44.675018
11851,132876003,Lower Chateaugay Lake,2.218336,399.0,4.300000e+12,390,6.000000e+13,221.800000,-74.018502,44.819012
11877,132876199,Mountain Pond,0.034300,0.0,4.300000e+12,390,6.000000e+13,3.430035,-74.042438,44.721654
11885,132876070,Bradley Pond,0.464830,0.0,4.300000e+12,390,6.000000e+13,46.480000,-73.887708,44.753069


In [2]:
import ee

ee.Initialize()

# Define the Landsat 5 bands and their corresponding standard names
LC8_BANDS = ['ST_B10']
STD_NAMES = ['temp_satellite']

# all_lakes=ee.FeatureCollection('projects/ee-mazarderakhsh/assets/ALAP-ALTM-Lakes-195')

def maskL8sr(image):
    # Define cloud, cloud shadow, and cirrus bitmasks
    cloud_shadow_bit_mask = 1 << 3
    cloud_cirrus_bit_mask = 1 << 2
    clouds_bit_mask = 1 << 4

    # Get the pixel QA band
    qa = image.select('QA_PIXEL')
    # Create masks for cloud, cloud shadow, and cirrus
    cloud_shadow_mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)
    clouds_mask = qa.bitwiseAnd(clouds_bit_mask).eq(0)
    cirrus_mask = qa.bitwiseAnd(cloud_cirrus_bit_mask).eq(0)

    # Combine masks to filter out cloudy, cloud shadow, and cirrus pixels
    mask = cloud_shadow_mask.And(clouds_mask).And(cirrus_mask)

    # Create saturation mask
    saturation_mask = image.select('QA_RADSAT').eq(0)
    waterMask= image.select('QA_PIXEL').eq(21952).Or(image.select('QA_PIXEL').eq(21824))

    # Apply scaling factors to optical bands
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    # Apply scaling factors to thermal bands
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(-124.15)

    # Replace original bands with scaled ones and apply masks
    return image.addBands(optical_bands, None, True) \
                .addBands(thermal_bands, None, True) \
                .updateMask(mask) \
                .updateMask(saturation_mask) \
                .updateMask(waterMask)



# Define a function to compute the mean reflectance values for the specified bands within the region of interest (lake)
def reflectance(img, lake):
    reflectance_values = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lake, scale=30).select(STD_NAMES)
    return img.set('DATE_SMP', img.date().format()).set('reflectance', reflectance_values)

# Initialize an empty list to store the dataframes for each lake
dfs = []

# df_coord['date'] = pd.to_datetime(df_coord['date'])
df_coord_filtered = df_coord[(df_coord['Lat'].notnull()) & (df_coord['Lon'].notnull())]

# Loop through each lake name and retrieve Landsat 5 imagery for that lake
for index, row in df_coord_filtered.iterrows():
    # Extract lake information
    pondId = row['PERMANENT_ID']
    pondname = row['GNIS_Name']
    Lon = row['Lon']
    Lat = row['Lat']
  


    lake_point = ee.Geometry.Point(row['Lon'], row['Lat'])

    #Create a 100-meter buffered box around the lake point
    lake_polygon = lake_point.buffer(90)
 
    # Retrieve Landsat 5 imagery for the specific lake
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filter(ee.Filter.calendarRange(1, 12, 'month')) \
        .filterBounds(lake_polygon) \
        .filter(ee.Filter.lt('CLOUD_COVER', 50)) \
        .map(maskL8sr) \
        .select(LC8_BANDS, STD_NAMES)

    
    # Map the reflectance function over the Landsat 5 ImageCollection for the specific lake
    map_reflectance = l8.map(lambda img: reflectance(img, lake_polygon))

    # Reduce the mapped image collection to get reflectance values for the specific lake
    list_reflectance = map_reflectance.reduceColumns(ee.Reducer.toList(2), ['DATE_SMP', 'reflectance']).values().get(0)

    # Convert the results to a pandas DataFrame
    df_reflectance = pd.DataFrame(list_reflectance.getInfo(), columns=['DATE_SMP', 'reflectance'])
    df_reflectance['DATE_SMP'] = pd.to_datetime(df_reflectance['DATE_SMP'])
    df_reflectance['DATE_SMP'] = df_reflectance['DATE_SMP'].dt.date
    df_reflectance['reflectance'] = df_reflectance['reflectance'].apply(lambda x: {k: v for k, v in x.items() if v is not None})

    # Unpack the 'reflectance' dictionary and create separate columns for each band
    df_reflectance = pd.concat([df_reflectance.drop('reflectance', axis=1),
                                df_reflectance['reflectance'].apply(pd.Series).astype('float64', errors='ignore')], axis=1)
    
    df_reflectance['SITE_ID'] = pondId
    df_reflectance['SITE_NAME'] = pondname
    df_reflectance['Lon'] = Lon
    df_reflectance['Lat'] = Lat

    # Add the DataFrame to the list
    dfs.append(df_reflectance)


# Concatenate all DataFrames into a single DataFrame
df_all_lakes_Landsat8 = pd.concat(dfs, ignore_index=True)

# Sort the DataFrame by 'DATE_SMP' in ascending order
df_all_lakes_Landsat8.sort_values(by='DATE_SMP', inplace=True)

# df_all_lakes.dropna(inplace=True)

df_all_lakes_Landsat8

,DATE_SMP,temp_satellite,SITE_ID,SITE_NAME,Lon,Lat
29792,2013-04-07,6.276268,89362311,Eagles Nest Pond,-73.593019,44.020329
253748,2013-04-07,4.244862,92083253,Bullpout Pond,-73.532747,44.006231
74111,2013-04-07,5.999931,89364639,Martin Pond,-73.971186,43.731939
253954,2013-04-07,4.244862,92083253,Bullpout Pond,-73.532747,44.006231
336505,2013-04-07,NaN,132876542,Lilypad Pond,-74.071009,44.776558
...,...,...,...,...,...,...
46241,2024-02-24,NaN,89363215,Flemings Pond,-73.627882,43.901868
65521,2024-02-24,NaN,47719607,Round Pond,-74.063924,43.643713
130884,2024-02-24,NaN,131841548,Thayer Lake,-74.832760,43.931976
217851,2024-02-24,-9.184595,92081791,Trout Pond,-73.572768,44.418518


In [3]:
# Remove duplicates from specified columns
df_all_lakes_Landsat8 = df_all_lakes_Landsat8.drop_duplicates(subset=['DATE_SMP', 'SITE_ID', 'SITE_NAME', 'Lon', 'Lat'])

# Now df_all_lakes_Landsat8 has duplicates removed from the specified columns
df_all_lakes_Landsat8

,DATE_SMP,temp_satellite,SITE_ID,SITE_NAME,Lon,Lat
29792,2013-04-07,6.276268,89362311,Eagles Nest Pond,-73.593019,44.020329
253748,2013-04-07,4.244862,92083253,Bullpout Pond,-73.532747,44.006231
74111,2013-04-07,5.999931,89364639,Martin Pond,-73.971186,43.731939
336505,2013-04-07,NaN,132876542,Lilypad Pond,-74.071009,44.776558
74005,2013-04-07,6.419744,89363747,Wilcox Pond,-73.708466,43.840196
...,...,...,...,...,...,...
29294,2024-02-24,-9.156501,89364071,Bullhead Pond,-74.242802,43.806685
320075,2024-02-24,NaN,132436138,Slender Pond,-74.850394,44.078869
150756,2024-02-24,NaN,131843539,Big Chief Pond,-74.829220,43.830216
65521,2024-02-24,NaN,47719607,Round Pond,-74.063924,43.643713


In [4]:
df_all_lakes_Landsat8.dropna(inplace=True)
df_all_lakes_Landsat8

C:\Users\Faculty\AppData\Local\Temp\ipykernel_19952\3376793513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_lakes_Landsat8.dropna(inplace=True)


,DATE_SMP,temp_satellite,SITE_ID,SITE_NAME,Lon,Lat
29792,2013-04-07,6.276268,89362311,Eagles Nest Pond,-73.593019,44.020329
253748,2013-04-07,4.244862,92083253,Bullpout Pond,-73.532747,44.006231
74111,2013-04-07,5.999931,89364639,Martin Pond,-73.971186,43.731939
74005,2013-04-07,6.419744,89363747,Wilcox Pond,-73.708466,43.840196
254266,2013-04-07,3.806666,115353849,Lost Pond,-73.712213,44.258692
...,...,...,...,...,...,...
97833,2024-02-24,-1.469739,50520965,Pickerel Pond,-73.847774,43.233088
175548,2024-02-24,-8.942218,131845150,Beaverdam Pond,-74.850027,43.699852
217747,2024-02-24,-9.184595,92081791,Trout Pond,-73.572768,44.418518
98040,2024-02-24,-7.022225,89361955,Stump Pond,-73.644170,44.074047


In [10]:
import pandas as pd


# Create an empty list to store the results
results = []

# Loop over each lake in the DataFrame
for lake_id, lake_data in df_all_lakes_Landsat8.groupby('SITE_ID'):
    pond_n = lake_data['SITE_NAME'].iloc[0]  # Get the lake name
    
    # Filter data for the specified years and summer months
    summer_data_2021_2023 = lake_data[
        (lake_data['DATE_SMP'].dt.year >= 2021) & 
        (lake_data['DATE_SMP'].dt.year <= 2023) & 
        (lake_data['DATE_SMP'].dt.month.isin([6, 7, 8]))
    ]
    
    # Calculate the average mean temperature for summer
    average_mean_temp = summer_data_2021_2023['temp_satellite'].mean()
    
    # Append the result to the results list
    results.append({
        'Lake_ID': lake_id,
        'Lake_name': pond_n,
        'Average_mean_temp': average_mean_temp
    })

# Create a DataFrame from the results
average_temps_df = pd.DataFrame(results)

# Save the results to a CSV file
average_temps_df.to_csv('average_mean_temps_landsat8_summer_2021_2023_4242024_WaterMask.csv', index=False)

print("Average mean temperatures saved.csv")


Average mean temperatures saved.csv


In [6]:
# Define the file path for the Excel file
excel_file_path = 'Landsat_8_1000Lakes_4242024_WaterMask.xlsx'

# Export the DataFrame to Excel
df_all_lakes_Landsat8.to_excel(excel_file_path, index=False)

print("DataFrame successfully exported to Excel file:", excel_file_path)

DataFrame successfully exported to Excel file: Landsat_8_1000Lakes_4242024_WaterMask.xlsx


In [7]:
import pandas as pd
from scipy import stats
import numpy as np


# Create a list to store the results
results = []
df_all_lakes_Landsat8['DATE_SMP'] = pd.to_datetime(df_all_lakes_Landsat8['DATE_SMP'])

# Loop over each lake in the DataFrame
for lake_id, lake_data in df_all_lakes_Landsat8.groupby('SITE_ID'):
    pond_n = df_all_lakes_Landsat8.loc[df_all_lakes_Landsat8['SITE_ID'] == lake_id, 'SITE_NAME'].iloc[0]

    # Combine data for June, July, and August
    summer_data = lake_data[lake_data['DATE_SMP'].dt.month.isin([1,2,3,4,5,6, 7, 8,9,10,11,12])]
    
    # Remove NaN or blank values from x_values and corresponding y_values
    x_values = summer_data['DATE_SMP'].values.astype(np.int64) // (10 ** 9)  # Convert to seconds
    y_values = summer_data['temp_satellite'].values
    mask = ~np.isnan(x_values) & ~np.isnan(y_values)
    x_values = x_values[mask]
    y_values = y_values[mask]
    
    # Check if x and y contain more than one distinct value
    if len(np.unique(x_values)) > 1:
        slope, _, r_value, p_value, _ = stats.linregress(x_values, y_values)
        
        # Check if the trend is significant (p-value < 0.05)
        if p_value < 0.05:
            slope_per_decade = slope * 10 * 365 * 24 * 3600  # Convert to °C/decade
            result = {
                'Lake_ID': lake_id,
                'Lake_name': pond_n,
                'Month': 'Annual',
                'Slope_per_Decade': slope_per_decade,
                'P-value': p_value,
                'R-value': r_value,
                'Temp_satellite': y_values.mean()  # Calculate mean temperature for non-blank values
            }
            results.append(result)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('significant_slopes_L8_1000_Annual_4242024_WaterMask.csv', index=False)

print("Results saved to significant_slopes_L8_1000_Annual_4122024_1.csv")


C:\Users\Faculty\AppData\Local\Temp\ipykernel_19952\1279970173.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_lakes_Landsat8['DATE_SMP'] = pd.to_datetime(df_all_lakes_Landsat8['DATE_SMP'])


Results saved to significant_slopes_L8_1000_Annual_4122024_1.csv


In [8]:
import pandas as pd
from scipy import stats
import numpy as np


# Create a list to store the results
results = []
df_all_lakes_Landsat8['DATE_SMP'] = pd.to_datetime(df_all_lakes_Landsat8['DATE_SMP'])

# Loop over each lake in the DataFrame
for lake_id, lake_data in df_all_lakes_Landsat8.groupby('SITE_ID'):
    pond_n = df_all_lakes_Landsat8.loc[df_all_lakes_Landsat8['SITE_ID'] == lake_id, 'SITE_NAME'].iloc[0]

    # Combine data for June, July, and August
    summer_data = lake_data[lake_data['DATE_SMP'].dt.month.isin([6, 7, 8])]
    
    # Remove NaN or blank values from x_values and corresponding y_values
    x_values = summer_data['DATE_SMP'].values.astype(np.int64) // (10 ** 9)  # Convert to seconds
    y_values = summer_data['temp_satellite'].values
    mask = ~np.isnan(x_values) & ~np.isnan(y_values)
    x_values = x_values[mask]
    y_values = y_values[mask]
    
    # Check if x and y contain more than one distinct value
    if len(np.unique(x_values)) > 1:
        slope, _, r_value, p_value, _ = stats.linregress(x_values, y_values)
        
        # Check if the trend is significant (p-value < 0.05)
        if p_value < 0.05:
            slope_per_decade = slope * 10 * 365 * 24 * 3600  # Convert to °C/decade
            result = {
                'Lake_ID': lake_id,
                'Lake_name': pond_n,
                'Month': 'June-August',
                'Slope_per_Decade': slope_per_decade,
                'P-value': p_value,
                'R-value': r_value,
                'Temp_satellite': y_values.mean()  # Calculate mean temperature for non-blank values
            }
            results.append(result)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('significant_slopes_L8_1000_SUMMER_4242024_WaterMask.csv', index=False)

print("Results saved to significant_slopes_L8_1000_SUMMER_4122024_1.csv")


C:\Users\Faculty\AppData\Local\Temp\ipykernel_19952\2129514215.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_lakes_Landsat8['DATE_SMP'] = pd.to_datetime(df_all_lakes_Landsat8['DATE_SMP'])


Results saved to significant_slopes_L8_1000_SUMMER_4122024_1.csv


In [9]:
import pandas as pd
from scipy import stats
import numpy as np

# Create a list to store the results
results = []
df_all_lakes_Landsat8['DATE_SMP'] = pd.to_datetime(df_all_lakes_Landsat8['DATE_SMP'])

# Loop over each lake in the DataFrame
for lake_id, lake_data in df_all_lakes_Landsat8.groupby('SITE_ID'):
    pond_n = df_all_lakes_Landsat8.loc[df_all_lakes_Landsat8['SITE_ID'] == lake_id, 'SITE_NAME'].iloc[0]

    # Combine data for June, July, and August
    summer_data = lake_data[lake_data['DATE_SMP'].dt.month.isin([5,6, 7, 8,9,10,11])]
    
    # Filter out rows with NaN values
    summer_data = summer_data.dropna(subset=['temp_satellite'])
    
    # Check if the data is not empty
    if not summer_data.empty:
        # Compute the linear regression
        x_values = summer_data['DATE_SMP'].values.astype(np.int64) // (10 ** 9)  # Convert to seconds
        y_values = summer_data['temp_satellite'].values
        
        # Check if x and y contain more than one value
        if len(x_values) > 1:
            slope, _, r_value, p_value, _ = stats.linregress(x_values, y_values)
            
            # Check if the trend is significant (p-value < 0.05)
            if p_value < 0.05:
                slope_per_decade = slope * 10 * 365 * 24 * 3600  # Convert to °C/decade
                result = {
                    'Lake_ID': lake_id,
                    'Lake_name': pond_n,
                    'Month': 'May-November',
                    'Slope_per_Decade': slope_per_decade,
                    'P-value': p_value,
                    'R-value': r_value,
                    'Temp_satellite': y_values.mean()  # Calculate mean temperature
                }
                results.append(result)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('significant_slopes_L8_1000_May_November_4242024_WaterMask.csv', index=False)

print("Results saved to significant_slopes_L8_1000_May_November_4122024_1.csv")

C:\Users\Faculty\AppData\Local\Temp\ipykernel_19952\101048976.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_lakes_Landsat8['DATE_SMP'] = pd.to_datetime(df_all_lakes_Landsat8['DATE_SMP'])


Results saved to significant_slopes_L8_1000_May_November_4122024_1.csv
